In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import LeaveOneOut
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import neighbors, metrics

In [2]:
#https://keras.io/api/applications/

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

#freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [3]:
def create_model(base_model):
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.1)(x)
    x = Dense(100, activation='relu')(x)
    #x = keras.layers.Dropout(0.3)(x)
    
    
    # let's add a fully-connected layer
    x = Dense(231, activation='softmax')(x)
    #x = keras.layers.Dropout(0.3)(x)
    
    # and a logistic layer
    #predictions = Dense(2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=x)
    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

In [4]:
def read_images(N, num_classes):

    #Declaring array of images 
    images = []

    #loop to import images from file directory
    for i in range (12): 
        x = np.asarray(Image.open("C:/Users/Mark Fahy/Desktop/Final Year/Final Year Project/bongard-problems-master/bongard-problems-master/p"+str(N)+"/"+str(i)+".png").convert('L'))
        x = np.array((x, x, x))
        #print(x.shape)
        images.append(x)
        
    #np.tile(images, 3)
    #Converting to numpy array
    dataset = np.array(images)
    dataset = dataset.reshape((-1, 104, 104, 3))    

    #Y data
    yvalues = (0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1)
    
    # Scale images to the [0, 1] range
    dataset = dataset.astype("float32") / 255
    

    # convert class vectors to binary class matrices
    #yvalues = keras.utils.to_categorical(yvalues, num_classes)    
    
    return dataset, yvalues

In [5]:
accuracies = []
losses = []
folds = 0
problems = 0

global_x_train = []
global_x_test = []
global_y_train = []
global_y_test = []

num_classes = 231
#input_shape = (104, 104, 1)

for i in range (1, 232): 
    dataset, yvalues = read_images(i, num_classes)
    #problems = problems + 1
    #print("Problem number: ", problems)
    
    x_train, x_test, y_train, y_test = train_test_split(dataset, yvalues, test_size=0.16666)  
    y_train = np.ones_like(y_train) * (i - 1)
    y_test = np.ones_like(y_test) * (i - 1)

    
    global_x_train.extend(x_train)
    global_x_test.extend(x_test)
    global_y_train.extend(y_train)
    global_y_test.extend(y_test)
    
global_x_train = np.array(global_x_train)
global_x_test = np.array(global_x_test)
global_y_train = np.array(global_y_train)
global_y_test = np.array(global_y_test)


# convert class vectors to binary class matrices
global_y_test = keras.utils.to_categorical(global_y_test, num_classes) 
global_y_train = keras.utils.to_categorical(global_y_train, num_classes) 


#print(global_x_train)
#print(global_y_train)
print(global_y_test.shape)
print(global_y_test)

(462, 231)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [6]:
model = create_model(base_model) 
model.fit(global_x_train, global_y_train, epochs = 5, validation_data = (global_x_test, global_y_test))

Epoch 1/5
73/73 [==============================] - 23s 264ms/step - loss: 5.4545 - accuracy: 0.0104 - val_loss: 5.3295 - val_accuracy: 0.0216
Epoch 2/5
73/73 [==============================] - 21s 284ms/step - loss: 5.0328 - accuracy: 0.0524 - val_loss: 4.9534 - val_accuracy: 0.0606
Epoch 3/5
73/73 [==============================] - 21s 288ms/step - loss: 4.3829 - accuracy: 0.1264 - val_loss: 4.5843 - val_accuracy: 0.1082
Epoch 4/5
73/73 [==============================] - 21s 293ms/step - loss: 3.8048 - accuracy: 0.1935 - val_loss: 4.4181 - val_accuracy: 0.1320
Epoch 5/5
73/73 [==============================] - 21s 293ms/step - loss: 3.3673 - accuracy: 0.2632 - val_loss: 4.3484 - val_accuracy: 0.1407


In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                           

In [24]:
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [25]:
x_network_output = new_model.predict(global_x_train)
x_network_output_test = new_model.predict(global_x_test)
print(x_network_output.shape)

(2310, 100)


In [26]:
accuracies = []
t_accuracies = []
num_classes = 2


for i in range (1, 21): 
    dataset, yvalues = read_images(i, num_classes)
    #problems = problems + 1
    #print("Problem number: ", problems)
    
    x_train, x_test, y_train, y_test = train_test_split(dataset, yvalues, test_size=0.16666)
    x_train_emb = new_model.predict(x_train)
    x_test_emb = new_model.predict(x_test)
        
    kNN_model = neighbors.KNeighborsClassifier(n_neighbors = 3)
    kNN_model.fit(x_train_emb, y_train)
    
    predictions_training = kNN_model.predict(x_train_emb)
    predictions_test = kNN_model.predict(x_test_emb)
    
    accuracy_training = metrics.accuracy_score(y_train, predictions_training)
    accuracy_test = metrics.accuracy_score(y_test, predictions_test)      
    
    accuracies.append(accuracy_training)
    t_accuracies.append(accuracy_test)       
        
        
print(accuracies)
print(t_accuracies)

print("Accuracy on training data: ", (sum(accuracies) / len(accuracies)) )
print("Accuracy on test data: ", (sum(t_accuracies) / len(t_accuracies)) )

[1.0, 0.8, 1.0, 0.7, 0.9, 0.7, 0.7, 0.9, 0.5, 0.9, 0.8, 0.8, 0.9, 0.9, 0.8, 0.7, 0.6, 0.8, 0.8, 0.6]
[1.0, 1.0, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0.0, 1.0, 0.5, 1.0, 0.0, 0.5, 1.0, 0.5]
Accuracy on training data:  0.7900000000000003
Accuracy on test data:  0.625


In [ ]:
from sklearn import neighbors, metrics

kNN_model = neighbors.KNeighborsClassifier()
kNN_model.fit(x_network_output, global_y_train)

predictions_training = kNN_model.predict(x_network_output)
predictions_test = kNN_model.predict(x_network_output_test)

accuracy_training = metrics.accuracy_score(global_y_train, predictions_training)
accuracy_test = metrics.accuracy_score(global_y_test, predictions_test)
print("Accuracy on training data:",accuracy_training)
print("Accuracy on test data:",accuracy_test)